In [1]:
%pip install mediapipe opencv-python


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mediapipe as mp
import cv2
import numpy as np


In [3]:
# Calculate the angle between three points using the cosine rule
def calculate_angle(a, b, c):


    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])

    radians = np.arccos(np.dot(b - a, c - b) / (np.linalg.norm(b - a) * np.linalg.norm(c - b)))
    angle = np.degrees(radians)

    return angle

In [4]:
def count_lateral_raises():
    mp_pose = mp.solutions.pose
    mp_drawing = mp.solutions.drawing_utils
    cap = cv2.VideoCapture(0)

    # cards constraints
    card_width, card_height = 150, 80
    card_x, card_y = 10, 10
    card_color = (255, 153, 13)

    with mp_pose.Pose(model_complexity =2 ,min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        raise_count = 0
        is_raising = False
        prev_angle = None
        prev_angle1 = None
        delta=None
        delta1=None

        while True:
            ret, image = cap.read()

            image = cv2.flip(image, 1)

            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            results = pose.process(image_rgb)

            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

            if results.pose_landmarks is not None:
                left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
                left_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_ELBOW]
                left_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_HIP]
                right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]
                right_elbow = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_ELBOW]
                right_hip = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_HIP]

                angle = calculate_angle(left_shoulder, left_elbow, left_hip)
            
                # cv2.putText(image, f"Right Angle: {angle:.2f} degrees", (10, 20), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)
                angle1 = calculate_angle(right_shoulder, right_elbow, right_hip)
                # cv2.putText(image, f"left Angle: {angle1:.2f} degrees", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

                # For lateral raise, we check the angle between the arms
                if prev_angle is not None:
                    delta = abs(angle - prev_angle)
                    print(delta)
                    delta1 = abs(angle1 - prev_angle1)
                    if not is_raising and delta > 15 and delta1 > 15:
                        is_raising = True
                    elif is_raising and delta < 7 and delta1 < 7:
                        is_raising = False

                prev_angle = angle
                prev_angle1 = angle1

                if is_raising and angle1>120 and angle > 120:
                    raise_count += 1

            cv2.rectangle(image, (card_x, card_y), (card_x + card_width, card_y + card_height), card_color, -1)

            # Display the text inside the card
            text_x, text_y = card_x + 10, card_y + 30
            text_line1 = 'Lateral Raise'
            text_line2 = f'Count: {raise_count}'
            line_spacing = 30
            font_scale = 0.7
            font_color = (255, 255, 255)

            cv2.putText(image, text_line1, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)
            cv2.putText(image, text_line2, (text_x, text_y + line_spacing), cv2.FONT_HERSHEY_COMPLEX, font_scale, font_color, 2, cv2.LINE_AA)

            cv2.imshow('Lateral Raise counter', image)

            if cv2.waitKey(1) == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

    return raise_count

In [5]:
count = count_lateral_raises()
print(count)

3.727926785330272
0.31979669128175914
4.631483265826077
0.13099669796450186
1.1826489198765202
0.42289040047786486
12.32561739987673
9.892663211185699
27.55974746098235
0.5150167267893266
27.003700663255714
5.514844642271868
1.7336537758295663
2.438391582055928
1.9479948005014478
14.672026657289123
0.7766599174724647
3.9059508746989806
3.0712097530492315
5.7197213633236075
1.539470616833711
2.9550598173799116
2.1350486685659646
0.6230305876448412
0.4259332518650538
0.4763033568173398
1.3320609259161813
0.5260908663542239
0.3886397896777396
5.779398832829251
7.581857255800241
1.6011161847889532
5.251857017748613
17.58219884896244
9.078599586000365
9.825353733888981
15.592611482152591
10.235125149526809
3.1400245014161072
10.680617521482105
2.448442975129609
2.8612639030732083
5.95034897072243
26.60332320826363
4.13629824812935
4.473357155486468
1.3525192241117594
30.95360287383683
11.934766082733077
2.45392187084812
10.341359995944984
2.125838199327376
2.7379785587668692
2.6526035839694